In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
main_character = "Lana Rhoades"

second_character = "Johnny Sins"

In [9]:

from transformers import pipeline

pipe = pipeline("text-generation", model="pranavpsv/genre-story-generator-v2")
text = pipe(f"{main_character} gets stuck in the washing machine, and {second_character} needs to get them unstuck")
text = text[0]["generated_text"].rstrip(".")
sentences = text.split(".")
if len(sentences[-1]) == 0:
    sentences.pop()
while len(sentences) < 4:
    more_text = pipe(sentences[-1])
    more_text = more_text[0]["generated_text"].rstrip(".")
    sentences.pop()
    sentences.extend(more_text.split("."))
    if len(sentences[-1]) == 0:
        sentences.pop()
sentences = sentences[:4]
if sentences[-1][-1] != ".":
    sentences[-1] = sentences[-1] + "..."
print(sentences)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Lana Rhoades gets stuck in the washing machine, and Johnny Sins needs to get them unstuck', " Johnny is the son of a wealthy Mississippi businessman called Jeff Davis, and they can't get their money back just yet, but when he catches them in a road rage and then shoots the driver (Brock Wilson at close range), Johnny learns that he's the killer and leaves him behind to die", ' Johnny was also responsible for helping his father build his own laundry plant in Arizona', '  Jeff...']


In [10]:
!pip install diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
ALLIGNMENT_LEFT = 0
ALLIGNMENT_CENTER = 1 
ALLIGNMENT_RIGHT = 2
ALLIGNMENT_TOP = 3
ALLIGNMENT_BOTTOM = 4

def text_box(text, image_draw, font, box, horizontal_allignment = ALLIGNMENT_LEFT, vertical_allignment = ALLIGNMENT_TOP, **kwargs):
    x = box[0]
    y = box[1]
    width = box[2]
    height = box[3]
    lines = text.split('\n')
    true_lines = []
    for line in lines:
        if font.getsize(line)[0] <= width:
            true_lines.append(line) 
        else:
            current_line = ''
            for word in line.split(' '):
                if font.getsize(current_line + word)[0] <= width:
                    current_line += ' ' + word 
                else:
                    true_lines.append(current_line)
                    current_line = word 
            true_lines.append(current_line)
    
    x_offset = y_offset = 0
    lineheight = font.getsize(true_lines[0])[1] * 1.2 # Give a margin of 0.2x the font height
    if vertical_allignment == ALLIGNMENT_CENTER:
        y = int(y + height / 2)
        y_offset = - (len(true_lines) * lineheight) / 2
    elif vertical_allignment == ALLIGNMENT_BOTTOM:
        y = int(y + height)
        y_offset = - (len(true_lines) * lineheight)
    
    for line in true_lines:
        linewidth = font.getsize(line)[0]
        if horizontal_allignment == ALLIGNMENT_CENTER:
            x_offset = (width - linewidth) / 2
        elif horizontal_allignment == ALLIGNMENT_RIGHT:
            x_offset = width - linewidth
        image_draw.text(
            (int(x + x_offset), int(y + y_offset)),
            line,
            fill=(0, 0, 0),
            font=font,
            **kwargs
        )
        y_offset += lineheight


In [12]:
from diffusers import DiffusionPipeline
from PIL import Image, ImageDraw, ImageFont, ImageColor, ImageChops

ldm = DiffusionPipeline.from_pretrained("hakurei/waifu-diffusion")
ldm = ldm.to('cuda')

images = []
for sentence in sentences:
    image = ldm(sentence, guidance_scale=6).images[0]
    images.append(image)

manga = Image.new("RGB", (1324, 1324))
manga = ImageChops.invert(manga)

manga.paste(images[0], (100, 100))
manga.paste(images[1], (712, 100))
manga.paste(images[2], (100, 712))
manga.paste(images[3], (712, 712))

draw = ImageDraw.Draw(manga)
draw.rectangle([(312, 487), (1012, 837)], fill=ImageColor.getrgb("white"))
text_box(text, draw, ImageFont.truetype("Pixellettersfull-BnJ5.ttf", 35), [325, 500, 674, 324])

manga.save("manga.png")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]